In [92]:
import os
import time
import openai

In [39]:
API_KEY_PATH = "/home/borjalozano/.openai_lozanoalvarezb_key"
with open(os.path.expanduser(API_KEY_PATH),'r') as file:
    API_KEY=file.read()

# openai.organization = "org-04VXSUT9JoL0BiDAs1kMZ9PN"
openai.api_key = API_KEY

In [40]:
# Test authentication
models = openai.Model.list()["data"]

In [22]:
openai.Completion.create(
  model="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

<OpenAIObject text_completion id=cmpl-6yiVczf1jLUsqre3UG7pGHA8ShGhc at 0x7fd21ecf93a0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test"
    }
  ],
  "created": 1679928792,
  "id": "cmpl-6yiVczf1jLUsqre3UG7pGHA8ShGhc",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

In [18]:
response = openai.Embedding.create(
    input="Your text string goes here",
    model="text-embedding-ada-002"
)
embeddings = response['data'][0]['embedding']

# Exploring topics using OpenAI-GPT

In [85]:
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',None)
pd.set_option('display.max_columns', None)


<div class="alert alert-block alert-info">
Download the parsed claims from https://drive.google.com/file/d/1QlCtKtSwdUX6NWOcUXijmxi4JoSdp49O/view?usp=sharing
</div>

In [3]:
claims_path = "../claims_parse_urls-6m-hard_clean.json"

In [4]:
claims = pd.read_json(claims_path)

In [125]:
config_id = "3f7502f069c028e2e9bb764130bb2c9f"
topics_path = f"topic_runs/{config_id}/topics_df.json"

In [9]:
topics_df = pd.read_json(topics_path)

In [11]:
claims = topics_df.join(claims)

In [69]:
fakes = claims[claims.cluster == 297].sort_values("probability",ascending=False)["claimReviewed_t"].head(5).values
prompt = " \n".join(fakes)

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": "Eres un Senior Data Scientist que aplica sus conocimientos para combatir la desinformación. Trabajas en un estudio sobre como esta se propaga. Sostienes una hipótesis: en ocasiones las mentiras aparecen en distintos sitios, adaptadas al contexto local. Defines la información común entre estos bulos como narrativa. Estas encargado de sintetizar varios bulos en una frase. Dicha frase ha de contenter la narrativa compartida entre todos los bulos. Es imprescindible que la frase contenga únicamente información que este presente en la mayoría de los bulos. Los bulos son: "},
      {"role": "user", "content": prompt}
    ],
    max_tokens=200,
    temperature=0.2
)

In [91]:
narratives = {}

In [106]:
for cluster in claims.cluster.unique():
    if cluster in narratives:
        continue

    fakes = claims[claims.cluster == cluster].sort_values("probability",ascending=False)["claimReviewed_t"].head(5).values
    prompt = "Eres un Senior Data Scientist que aplica sus conocimientos para combatir la desinformación. Trabajas en un estudio sobre como esta se propaga. Sostienes una hipótesis: en ocasiones las mentiras aparecen en distintos sitios, adaptadas al contexto local. Defines la información común entre estos bulos como narrativa. Estas encargado de sintetizar varios bulos en una frase. Dicha frase ha de contenter la narrativa compartida entre todos los bulos. Es imprescindible que la frase contenga únicamente información que este presente en la mayoría de los bulos. Si los bulos no comparten suficiente información para definir una narrativa escribe \"NONE\". Los bulos son: "
    fakes = " \n".join(fakes)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": prompt},
            {"role": "user", "content": fakes}
        ],
        max_tokens=200,
        temperature=0.2
    )
    narratives[cluster] = response["choices"][0]["message"]["content"]
    print(f"{cluster}: {narratives[cluster]}\n")

471: La frase que contiene la narrativa compartida entre todos los bulos es: "Esta galleta acaba de llegar a Brasil. Viene con 2 pastillas adentro que causan parálisis".

137: Narrativa compartida: Oxford aprueba un plan de prueba de 'bloqueo climático' para reducir las emisiones de carbono en 2024.

Frase sintetizada: Oxford aprueba plan de prueba de 'bloqueo climático' para reducir emisiones de carbono en 2024.

353: La narrativa compartida entre estos bulos es la presencia de asesinos en serie en diferentes ciudades de Estados Unidos. Por lo tanto, la frase que sintetiza esta narrativa podría ser: "Varios asesinos en serie están activos en diferentes ciudades de Estados Unidos".

274: Narrativa: Residentes de Moldavia bloquean la entrada de una columna de equipos militares de la OTAN.

236: NONE. No hay información común entre los bulos que permita definir una narrativa compartida.

146: La narrativa compartida entre estos bulos es que Lula obtuvo una cantidad de votos significativa

In [143]:
narratives_df = pd.Series(narratives,name="Narrative").sort_index().apply(lambda x: None if "NONE" in x else x)
narratives_df = narratives_df.sort_index().reset_index()
narratives_df = narratives_df.rename({"index":"cluster"},axis=1).set_index("cluster")

In [144]:
narratives_df.to_json(f"topic_runs/{config_id}/narratives.json")

In [151]:
narratives_df[narratives_df.Narrative.notna()].sample(3)

,Narrative
cluster,
480,La narrativa compartida entre estos bulos es que JP Nadda comparó al gobierno del BJP con violadores.
311,La narrativa compartida entre los bulos es la utilización de autobuses para transportar simpatizantes o manifestantes a eventos políticos o protestas.
97,Narrativa: Elon Musk planea adquirir Meta e Instagram.


In [159]:
narratives_df.reset_index()

,cluster,Narrative
0,-1,None
1,0,None
2,1,La narrativa compartida entre todos los bulos es que el cambio climático no es causado por las emisiones humanas de CO2.
3,2,None
4,3,None
...,...,...
479,478,"La narrativa compartida entre los bulos es que un hombre disfrazado de chamán, inspirado en el ""Chamán"" del Capitolio en Estados Unidos, participó en el asalto al Congreso en Brasil el 8 de enero de 2023."
480,479,"La narrativa compartida entre todos los bulos es la curación de enfermedades mediante productos aprobados por la FDA o agencias de salud en distintos países, como Cagan Nano, Bio Stomach Curcumin, Thyroid Mana Nano Curcumin y la leche de curcumina Hera Nano."
481,480,La narrativa compartida entre estos bulos es que JP Nadda comparó al gobierno del BJP con violadores.
482,481,None


In [174]:
pd.merge(claims,narratives_df,on="cluster",how="left").sort_values("cluster")[["_id","cluster","Narrative","country","claimReviewed","claimReviewed_t"]].to_csv("borja.csv")